In [2]:
!pip install gensim
!pip install transformers

     |████████████████████████████████| 4.0 MB 4.4 MB/s 
     |████████████████████████████████| 6.6 MB 10.6 MB/s 
     |████████████████████████████████| 895 kB 40.6 MB/s 
     |████████████████████████████████| 596 kB 33.5 MB/s 
     |████████████████████████████████| 77 kB 4.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
!ls

drive  sample_data


In [4]:
import os

os.chdir('drive/MyDrive/William_2022')

In [5]:
import gensim
import torch
from transformers import BertTokenizer, BertModel
import pandas as pd
import numpy as np

import logging
logging.basicConfig(level=logging.INFO) 

import matplotlib.pyplot as plt
% matplotlib inline

In [6]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

## Preprocessing


In [ ]:
data_dir = 'DATASETS/CSV/transcript_emotion_6class'

first = 1
second = 2

transcript = pd.read_csv(data_dir + f'/p{first}.p{second}a.csv')
transcript.rename(columns = {'results/alternatives/0/transcript' : 'full_transcript'}, inplace = True) 
transcript.rename(columns = {'results/alternatives/0/words/0/word' : 'word'}, inplace = True) 

only_fulltranscript = transcript[transcript['new_words'].notna()]
only_fulltranscript = only_fulltranscript.reset_index()
only_fulltranscript = only_fulltranscript.iloc[:, 28:]

In [ ]:
word_list0 = []
word_list1 = []

# for j in range(0, len(df)):
for i in range(0, len(transcript)-1):
    if (transcript.seconds[i] == transcript.seconds[i+1]):
        #print(df.iloc[:i+2, 5])
        #print(df.word[i+1])
        word_list = transcript.word[i]
        #print(word_list)
        word_list0.append(word_list)
        
        
        #print(word_list0)
        
        
        
    else:
        word_list0.append(transcript.word[i])
        word_list1.append(word_list0)
        
        word_list0=[]

word_list1.append(word_list0)
transcript['new_words'] = ''

for i in range(0, len(word_list1)):
  #print(i)
  word_list1[i] = ' '.join(word_list1[i])
  transcript.new_words[i] = word_list1[i]

transcript.to_csv(f'p{first}.p{second}.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
transcript = pd.read_csv(f'p{first}.p{second}.csv')
transcript[f'p{first}_trans'] = ''
transcript['seconds_trans'] = ''
transcript['arousal_trans'] = ''
transcript['valence_trans'] = ''
transcript['happy_trans'] = ''
transcript['angry_trans'] = ''
transcript['nervous_trans'] = ''
transcript['sad_trans'] = ''

transcript[f'p{second}_trans'] = ''
transcript['seconds2_trans'] = ''
transcript['arousal2_trans'] = ''
transcript['valence2_trans'] = ''
transcript['happy2_trans'] = ''
transcript['angry2_trans'] = ''
transcript['nervous2_trans'] = ''
transcript['sad2_trans'] = ''
fir_p = pd.read_csv(f'DATASETS/CSV/averaged_emotions/p{first}.csv')
sec_p = pd.read_csv(f'DATASETS/CSV/averaged_emotions/p{second}.csv')

for i in range(0,len(word_list1)):
  
  transcript.iloc[i, 28:35] = fir_p.iloc[i]

  
for j in range(0, len(word_list1)):  
  transcript.iloc[j, 36:43] = sec_p.iloc[j]


transcript.to_csv(f'p{first}.p{second}a.csv')

In [ ]:
data_dir = 'DATASETS/CSV/transcript_emotion_6class'

first = 1
second = 2


transcript = pd.read_csv(data_dir + f'/p{first}.p{second}a.csv')
transcript.rename(columns = {'results/alternatives/0/transcript' : 'full_transcript'}, inplace = True) 
transcript.rename(columns = {'results/alternatives/0/words/0/word' : 'word'}, inplace = True) 

only_fulltranscript = transcript[transcript['new_words'].notna()]
only_fulltranscript = only_fulltranscript.reset_index()
only_fulltranscript = only_fulltranscript.iloc[:, 28:]


In [ ]:
only_fulltranscript

,index,Unnamed: 0,Unnamed: 0.1,results/alternatives/0/confidence,full_transcript,results/alternatives/0/words/0/confidence,reult/alternative/0/word/0/endTime,startTime,word,results/channelTag,...,nervous_trans,sad_trans,p32_trans,seconds2_trans,arousal2_trans,valence2_trans,happy2_trans,angry2_trans,nervous2_trans,sad2_trans
0,0,0,0,0.884961,okay the first are maybe you cannot understand...,0.912839,3.1,2.5,okay,31,...,1.333,1.0,NaN,5.0,3.667,3.333,2.000,1.000,1.667,1.0
1,1,1,1,0.884961,NaN,0.912839,4.4,4.3,the,31,...,1.333,1.0,NaN,10.0,3.667,3.333,2.000,1.000,1.667,1.0
2,2,2,2,0.884961,NaN,0.912839,4.8,4.4,first,31,...,1.333,1.0,NaN,15.0,4.000,4.000,2.000,1.000,1.333,1.0
3,3,3,3,0.884961,NaN,0.912839,5.1,4.8,are,31,...,1.333,1.0,NaN,20.0,4.000,4.000,2.000,1.000,1.333,1.0
4,4,4,4,0.884961,NaN,0.912839,6.8,6.5,maybe,31,...,1.333,1.0,NaN,25.0,3.333,3.000,1.667,1.333,1.667,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,111,111,111,0.863448,NaN,0.912839,74.1,74.0,in,31,...,1.000,1.0,NaN,560.0,3.333,3.000,1.333,1.333,1.333,1.0
112,112,112,112,0.863448,NaN,0.644002,74.8,74.1,myself,31,...,1.000,1.0,NaN,565.0,3.667,3.000,1.333,1.333,1.333,1.0
113,113,113,113,0.880028,yeah and finally from a legal point of view K...,0.912839,77.0,76.6,yeah,31,...,1.000,1.0,NaN,570.0,3.667,3.000,1.333,1.333,2.000,1.0
114,114,114,114,0.880028,NaN,0.912839,78.3,77.8,and,31,...,1.000,1.0,NaN,575.0,3.333,3.000,1.333,1.333,2.000,1.0


In [ ]:
full_transcript = pd.DataFrame()

In [ ]:
data_dir = 'DATASETS/CSV/transcript_emotion_6class/'

first = 1
second = 2



for csv in range(0, len(os.listdir(data_dir))):
  #print(csv)
  transcript = pd.read_csv(f'{data_dir}/{os.listdir(data_dir)[csv]}')
  #print(csv)
  #transcript = pd.read_csv(data_dir + f'/p{first}.p{second}a.csv')
  #print(transcript)
  only_fulltranscript = transcript[transcript['new_words'].notna()]
  only_fulltranscript = only_fulltranscript.reset_index()
  only_fulltranscript_fir = only_fulltranscript[['new_words',	'seconds_trans',	'arousal_trans',	'valence_trans',	'happy_trans',	'angry_trans',	'nervous_trans',	'sad_trans']]
  only_fulltranscript_sec = only_fulltranscript[['seconds2_trans',	'arousal2_trans',	'valence2_trans',	'happy2_trans',	'angry2_trans',	'nervous2_trans',	'sad2_trans']]
  only_fulltranscript_concat = pd.concat([only_fulltranscript_fir, only_fulltranscript_sec], axis = 1)
  #print(only_fulltranscript_fir)
  #print(only_fulltranscript_sec)
  full_transcript = full_transcript.append(only_fulltranscript_concat)
  #print(full_transcript2)
  
  #print(full_transcript2)
  first +=1
  second +=1



  
full_transcript.to_csv('full_transcript.csv',sep = ',', index = False)




## Word Embeddings

In [7]:
full_transcript = pd.read_csv('DATASETS/CSV/full_transcript_concat.csv')

In [15]:
full_transcript

,new_words,seconds_trans,arousal_trans,valence_trans,happy_trans,angry_trans,nervous_trans,sad_trans,seconds2_trans,arousal2_trans,valence2_trans,happy2_trans,angry2_trans,nervous2_trans,sad2_trans
0,all right first one the state for this accepti...,5.0,1.333,2.667,1.667,1.0,1.333,1.0,5.0,3.000,2.667,1.333,1.000,1.667,1.0
1,arrived in this especially in many Arab Jesus ...,10.0,1.667,2.667,1.667,1.0,1.333,1.0,10.0,3.333,3.000,1.333,1.000,2.000,1.0
2,Jeju Allah first of all I want to mention that...,15.0,2.000,2.667,1.333,1.0,1.333,1.0,15.0,3.333,2.667,1.333,1.000,2.333,1.0
3,movement in South Korea against against this y...,20.0,2.000,3.000,1.333,1.0,1.333,1.0,20.0,3.667,2.667,1.333,1.000,2.333,1.0
4,pretty much standard phobic activity because,25.0,2.000,3.000,1.333,1.0,1.333,1.0,25.0,3.333,2.667,1.333,1.000,2.333,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1858,The Courier what curse income,560.0,3.333,4.000,1.333,1.0,1.000,1.0,560.0,3.333,3.000,1.333,1.333,1.333,1.0
1859,cannot there with this issue and cannot,565.0,3.333,4.000,1.333,1.0,1.000,1.0,565.0,3.667,3.000,1.333,1.333,1.333,1.0
1860,change the no in this issue yeah,570.0,3.333,4.000,1.333,1.0,1.000,1.0,570.0,3.667,3.000,1.333,1.333,2.000,1.0
1861,your suggestion is very proper but there's no ...,575.0,3.333,4.000,1.333,1.0,1.000,1.0,575.0,3.333,3.000,1.333,1.333,2.000,1.0


In [34]:
x_train = tokenizer(
    text=full_transcript.new_words.tolist(),
    add_special_tokens=True,
    max_length=512,
    truncation=True,
    padding=True, 
    return_tensors='pt',
    return_token_type_ids = True,
    return_attention_mask = True,
    verbose = True)

In [35]:
x_train['input_ids'][0]

tensor([  101,  2035,  2157,  2034,  2028,  1996,  2110,  2005,  2023, 10564,
         2023,  8711,   102,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0])

In [30]:
x_train['attention_mask']

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])

In [ ]:
full_transcript

In [ ]:
full_transcript_words = full_transcript.new_words.tolist()

In [ ]:
full_transcript_words

In [ ]:
def one_hot_encoder(df):
    one_hot_encoding = []
    for i in tqdm(range(len(df))):
        temp = [0]*n_labels
        label_indices = df.iloc[i][2:8]
        for index in label_indices:
            temp[index] = 1
        one_hot_encoding.append(temp)
    return pd.DataFrame(one_hot_encoding)

In [ ]:
full_transcript['arousal_trans'].unique()

array([1.333, 1.667, 2.   , 2.667, 2.333, 3.   , 3.333, 3.667, 4.   ,
       4.333])

In [8]:
tokenized_text_list = []

indexed_tokens_list = []

segment_ids_list = []

tokens_tensor_list = []

segment_tensors_list = []

# def preprocess_text(only_fulltranscript):
for i in range(0, len(full_transcript)):
  text = full_transcript.new_words[i]
  marked = '[CLS]' + text + '[SEP]'

  tokenized_text = tokenizer.tokenize(marked)
  tokenized_text_list.append(tokenized_text)
  #print(tokenized_text)

  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
  indexed_tokens_list.append(indexed_tokens)

  # for tup in zip(tokenized_text, indexed_tokens):
  #   print('{:<12} {:>6,}'.format(tup[0], tup[1]))

  segment_ids = [i] * len(tokenized_text)
  segment_ids_list.append(segment_ids)
  #print(segment_ids)
  
  tokens_tensor = torch.tensor([indexed_tokens])
  tokens_tensor_list.append(tokens_tensor)

  segment_tensors = torch.tensor([segment_ids])
  segment_tensors_list.append(segment_tensors)

  # return tokenized_text, indexed_tokens, segment_ids, tokens_tensor, segment_tensors


In [12]:
segment_tensors_list

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

In [13]:
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased', 
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [ ]:
train_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:511], train_texts))

In [ ]:
def bert_text_preparation(text, tokenizer):
    
    marked_text = "[CLS] " + text + " [SEP]"
    tokenized_text = tokenizer.tokenize(marked_text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1]*len(indexed_tokens)

    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    return tokenized_text, tokens_tensor, segments_tensors



def get_bert_embeddings(tokens_tensor, segments_tensors, model):

    
    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)
       
        hidden_states = outputs[2][1:]


    token_embeddings = hidden_states[-1]
    token_embeddings = torch.squeeze(token_embeddings, dim=0)
    list_token_embeddings = [token_embed.tolist() for token_embed in token_embeddings]

    return list_token_embeddings

In [ ]:
target_word_embeddings = []
for text in only_fulltranscript.new_words:
    tokenized_text, tokens_tensor, segments_tensors = bert_text_preparation(text, tokenizer)
    list_token_embeddings = get_bert_embeddings(tokens_tensor, segments_tensors, model)
    #print(tokenized_text)
    # word_index = tokenized_text.index(#word)
    # word_embedding = list_token_embeddings[word_index]

    # target_word_embeddings.append(word_embedding)

In [ ]:
len(list_token_embeddings)

9

In [ ]:
tokens_tensor

tensor([[ 101, 3132, 3100, 3325, 2003, 2058,  102]])

In [ ]:
segments_tensors

tensor([[1, 1, 1, 1, 1, 1, 1]])

In [ ]:
list_token_embeddings